## Steps Summary

In [0]:
# 1. Mount google drive
# 2. Set the directory path of google drive
# 3. Load and Import all modules and libraries
# 4. Set Device
# 5. Import all the functions
# 6. Load transformations
# 7. Load dataset
# 8. Load Classes
# 9. Display sample images
# 10. Load Resnet18 Model
# 11. Define Loss function and optimizer
# 12. Run train and test
# 13. Grad-CAM
# 14. Misclassified images

### 1. Mount google drive

In [0]:
# Load data from Google drive
from google.colab import drive
drive.mount('/content/drive')

### 2. Set the directory path of google drive

In [0]:
import sys

sys.path.append('/content/drive/My Drive/eva-4/assignment-11')
sys.path.append('/content/drive/My Drive/eva-4/assignment-11/models')
sys.path.append('/content/drive/My Drive/eva-4/assignment-11/gradcam')
sys.path.append('/content/drive/My Drive/eva-4/assignment-11/sampleimages')
sys.path.append('/content/drive/My Drive/eva-4/assignment-11/misclassified')

In [0]:
cd '/content/drive/My Drive/eva-4/assignment-11'

In [0]:
ls

### 3. Load and Import all modules and libraries

In [0]:
!pip install torchsummary

In [0]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

In [0]:
!pip install torch-lr-finder -v --global-option="amp"

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torchvision.utils import make_grid
from torchsummary import summary

In [0]:
# Define hyper paramteres and constants
EPOCHS=24
trainaccuracies = []
trainlosses = []
testlosses = []
testaccuracies = []
lr_values = []

### 4. Set Device

In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

###  5. Import all the functions

In [0]:
from model import Net, ResNet, resnet18
from s11model import S11Model
from S11_functions import myfunc, transformations, loadcifar10dataset, getclasses, display, train, test, imshow, loadimage, saveimage
from S11_functions import getloss, getoptimizer, getscheduler
from gradcam import GradCAM, GradCAMpp, applygradcam
from gradcam_utils import Normalize, visualize_cam
from learningrate import findandplotlearningrate, plotCyclicLR, performLRRangeTest, getSchdeduler, getlr
from plotutils import plotimages, plotmisclassifiedimages, savemisclassifiedimages, plotmisclassifiedgradcamimages, plotmetrics 

###  6. Load Transformations

In [0]:
transform_train, transform_test = transformations()

###  7. Load Dataset

In [0]:
trainset, trainloader, testset, testloader = loadcifar10dataset(transform_train, transform_test)
print(trainloader)

###  8. Load Classes

In [0]:
classes = getclasses()
print(classes)

### 9. Display sample images

###### Sample Training Images

In [0]:
plotimages(device, classes, trainloader, 20)

###### Sample Test Images

In [0]:
plotimages(device, classes, testloader, 20)

### 10. Load Custom Resnet Model


In [0]:
print("Building Model")
net = S11Model()

In [0]:
# Display Model Summary
model = net.to(device)
summary(model, input_size=(3, 32, 32))

### 11. Define loss function and optimizer


In [0]:
criterion = getloss()

#### Find Learning Rate

##### Plot CyclicLR 

In [0]:
plotCyclicLR(total_iterations=10000, min_lr=0.005, max_lr=0.05, step_size=1000)

In [0]:
optimizer = getoptimizer(net, lr=1e-7, momentum=0.9, nesterov=True, weight_decay=0.05)

In [0]:
max_lr = performLRRangeTest(model, optimizer, criterion, device, trainloader, end_lr=100, num_iter=100)

In [0]:
scheduler = getSchdeduler(optimizer, max_lr = 0.0125, epochs=EPOCHS, steps_per_epoch=1, pct_start=0.25)

### 12. Run Train and Test

In [0]:
for epoch in range(1, EPOCHS + 1):
    epochlr = getlr(optimizer)
    print("\nEPOCH:", epoch,'LR:',epochlr)
    lr_values.append(epochlr)
 
    train(net, trainloader, device, optimizer, criterion, trainaccuracies, trainlosses, epoch)
    scheduler.step()
    testloss = test(net, testloader, device, criterion, testaccuracies, testlosses, epoch)

In [0]:
plt.plot(lr_values)

#### Plot Metrics - Accuracy and Loss

In [0]:
plotmetrics(trainaccuracies, testaccuracies, trainlosses, testlosses, savefilename="metrics")

### 14. Misclassified images

#### Get Misclassified Images

In [0]:
misclassifieddirectory = '/content/drive/My Drive/eva-4/assignment-11/misclassified'
names, titles = savemisclassifiedimages(model, device, classes, testloader, misclassifieddirectory, 25)

### 15. Analysis
- No of Epochs : 24
- Batch size : 512
- Best Test  Acc: 91.04%
- Best Train Acc: 98.03%
- Custom Model - S11Model
- Cutout : Yes
- Albumentations: Yes
- Hyperparameters
  - Optimizer: SGD
  - Learning rate: 0.0125
  - Momentum: 0.9